<a href="https://colab.research.google.com/github/gibranfp/CursoAprendizajeProfundo/blob/master/notebooks/04b_transferencia_flores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendizaje por transferencia para el reconocimiento de flores

In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.layers import Add, ZeroPadding2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, Activation
from tensorflow.keras.utils import to_categorical
import tensorflow_datasets as tfds


np.random.seed(1)
tf.random.set_seed(2)

## Conjunto de datos
Vamos a hacer aprendizaje por transferencia la red neuronal convolucional [MobileNetv2](https://arxiv.org/abs/1801.04381) para el reconocimiento de especies de flores usando el conjunto de imágenes [Flowers 102](https://www.robots.ox.ac.uk/~vgg/data/flowers/102/), el cual está compuesto por imágenes a color de 102 especies diferentes de flores. 

_Tensorflow Datasets_ tiene funciones para descargar este conjunto de imágenes: 

In [0]:
train = tfds.load(name='oxford_flowers102:2.*.*', with_info=False, as_supervised=True, split='train[:90%]')
valid = tfds.load(name='oxford_flowers102:2.*.*', with_info=False, as_supervised=True, split='train[-10%:]')
test = tfds.load(name='oxford_flowers102:2.*.*', with_info=False, as_supervised=True, split='test')


## Acrecentamiento de datos
Ahora vamos a definir nuestra tubería de datos para el entrenamiento, la cual tiene una función de mapeo que aplica ciertas transformaciones a las imágenes para incrementar su variedad. 

In [0]:
def procesa_imagen(imagen, categoria, imsize=(160,160)):
    imagen = tf.cast(imagen, 'float64')
    imagen = tf.image.resize(imagen, imsize)
    imagen /= 255.0
    imagen = tf.image.random_flip_left_right(imagen)
    imagen = tf.image.random_flip_up_down(imagen)
    imagen = tf.image.random_crop(imagen, size=[imsize[0], imsize[1], 3])
    imagen = tf.image.rot90(imagen, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    imagen = tf.clip_by_value(imagen, 0, 1)
    
    return imagen, categoria

ent_ds = train.map(procesa_imagen, num_parallel_calls=8)
ent_ds = ent_ds.batch(64)
ent_ds = ent_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

for i, (x, y_true) in enumerate(ent_ds, start=1):
    plt.subplot(2, 5, i)
    plt.imshow(x[0])
    plt.axis('off')
    if i == 10:
        break

Para el conjunto de validación y de prueba la tubería de datos no hace acrecentamiento de datos, solo redimensiona y normaliza las imágenes.

In [0]:
def carga_una_imagen(imagen, categoria):
    imagen = tf.cast(imagen, 'float64')
    imagen = tf.image.resize(imagen,(160,160))
    imagen /= 255.0
    return imagen, categoria
  
val_ds = valid.map(carga_una_imagen, num_parallel_calls=4)
val_ds = val_ds.batch(1)

## Definición de la red neuronal convolucional
Ahora vamos a definir nuestra red neuronal convolucional. Tomaremos de base MobileNetv2, la cual es una red neuronal convolucional que ocupa convoluciones separables en profundidad. En particular, usaremos las capas convolucionales preentrenadas en el conjunto de imágenes de [ImageNet](http://www.image-net.org/), a las cuales le agregaremos capas de submuestreo y capas densas como etapa de clasificación. 

In [0]:
class CNN(tf.keras.Model):
  def __init__(self):
    super(CNN, self).__init__()
    self.base = tf.keras.applications.MobileNetV2(input_shape=(160,160,3),
                                                  include_top=False,
                                                  weights='imagenet')
    self.gap = GlobalAveragePooling2D()
    self.dp = Dropout(0.7)
    self.sm = Dense(102, 
                    activation='softmax', 
                    kernel_regularizer='l2', 
                    bias_regularizer='l2')

  def call(self, x): 
    x = self.base(x)
    x = self.gap(x)
    x = self.dp(x)
    x = self.sm(x)
    
    return x

Construyamos nuestra red y visualicemos sus características:

In [0]:
modelo = CNN()
modelo.build(input_shape=(None, 160, 160 , 3))
modelo.summary()

Ahora definimos las funciones para realizar la propagación hacia adelante y la retropropagación en los ejemplos de entrenamiento y la propagación hacia adelante para los ejemplos de validación y de prueba.

In [0]:
@tf.function
def paso_ent(x, y, modelo, fn_perdida, optimizador):
    with tf.GradientTape() as cinta:
        y_pred = modelo(x)
        perdida = fn_perdida(y, y_pred)
    gradientes = cinta.gradient(perdida, modelo.trainable_variables)
    optimizador.apply_gradients(zip(gradientes, modelo.trainable_variables))
    return y_pred
  
@tf.function
def paso_prueba(imagen, modelo):
  return modelo(imagen)

## Entrenamiento preliminar
Ya podemos entrenar nuestra red. Primero solo ajustaremos los pesos y sesgos de la etapa de clasificación, por lo que congelaremos los de las capas convolucionales. Como función de pérdida usaremos la función de entropía cruzada categórica:

In [0]:
modelo.base.trainable = False
fn_perdida = tf.keras.losses.SparseCategoricalCrossentropy()
optimizador = tf.keras.optimizers.Adam(learning_rate=1e-3)

perdida_ent = tf.keras.metrics.SparseCategoricalCrossentropy()
exactitud_ent = tf.keras.metrics.SparseCategoricalAccuracy()

perdida_val = tf.keras.metrics.SparseCategoricalCrossentropy()
exactitud_val = tf.keras.metrics.SparseCategoricalAccuracy()

hist_perdida_ent = []
hist_exactitud_ent = []

hist_perdida_val = []
hist_exactitud_val = []

for epoca in range(20):
    for paso, (x, y) in enumerate(ent_ds):
        y_pred = paso_ent(x, y,  modelo, fn_perdida, optimizador)
        perdida_ent(y, y_pred)
        exactitud_ent(y, y_pred)
        
    for (x, y) in val_ds:
        y_pred = paso_prueba(x, modelo)
        perdida_val(y, y_pred)
        exactitud_val(y, y_pred)
    
    perdida_ent_res = perdida_ent.result().numpy()
    exactitud_ent_res = exactitud_ent.result().numpy() * 100
    perdida_ent.reset_states()
    exactitud_ent.reset_states()
    hist_perdida_ent.append(perdida_ent_res)
    hist_exactitud_ent.append(exactitud_ent_res)

    perdida_val_res = perdida_val.result().numpy()
    exactitud_val_res = exactitud_val.result().numpy() * 100
    perdida_val.reset_states()
    exactitud_val.reset_states()
    hist_perdida_val.append(perdida_val_res)
    hist_exactitud_val.append(exactitud_val_res)
  
    print('E{:2d} CCE(E)={:6.2f}, Exactidud(E)={:6.2f} CCE(V)={:6.2f} Exactitud(V)={:6.2f}'.format(epoca, 
                                                                                                   perdida_ent_res, 
                                                                                                   exactitud_ent_res,
                                                                                                   perdida_val_res, 
                                                                                                   exactitud_val_res))

## Ajuste fino
Después de ajustar por 5 épocas nuestras capas de clasificación, descongelaremos algunas de las últimas capas convolucionales y reanudaremos el entrenamiento por 5 épocas más. A este proceso se le conoce como ajuste fino.

In [0]:
modelo.base.trainable = True # descongela pesos y sesgos de todas las capas
for layer in modelo.base.layers[:130]: # congela de nuevo primeras 130 capas
  layer.trainable =  False

for epoca in range(20,40):
    for paso, (x, y) in enumerate(ent_ds):
        y_pred = paso_ent(x, y,  modelo, fn_perdida, optimizador)
        perdida_ent(y, y_pred)
        exactitud_ent(y, y_pred)
        
    for (x, y) in val_ds:
        y_pred = paso_prueba(x, modelo)
        perdida_val(y, y_pred)
        exactitud_val(y, y_pred)
    
    perdida_ent_res = perdida_ent.result().numpy()
    exactitud_ent_res = exactitud_ent.result().numpy() * 100
    perdida_ent.reset_states()
    exactitud_ent.reset_states()
    hist_perdida_ent.append(perdida_ent_res)
    hist_exactitud_ent.append(exactitud_ent_res)

    perdida_val_res = perdida_val.result().numpy()
    exactitud_val_res = exactitud_val.result().numpy() * 100
    perdida_val.reset_states()
    exactitud_val.reset_states()
    hist_perdida_val.append(perdida_val_res)
    hist_exactitud_val.append(exactitud_val_res)
  
    print('E{:2d} CCE(E)={:6.2f}, Exactidud(E)={:6.2f} CCE(V)={:6.2f} Exactitud(V)={:6.2f}'.format(epoca, 
                                                                                                   perdida_ent_res, 
                                                                                                   exactitud_ent_res,
                                                                                                   perdida_val_res, 
                                                                                                   exactitud_val_res))

## Gráficas de la pérdida y la exactitud

Visualicemos la evolución de la pérdida de entropía cruzada y la exactitud durante el entrenamiento:

In [0]:
plt.figure(1)
plt.plot(hist_perdida_ent, color='red', label='Entrenamiento')
plt.plot(hist_perdida_val, color='blue', label=u'Validación')
plt.xlabel(u'Época')
plt.ylabel(u'Pérdida')
plt.legend(loc='upper right')

plt.figure(2)
plt.plot(hist_exactitud_ent, color='red', label='Entrenamiento')
plt.plot(hist_exactitud_val, color='blue', label=u'Validación')
plt.xlabel(u'Época')
plt.ylabel(u'Exactitud')
plt.legend(loc='upper right')

plt.show()

## Evaluación

Ya entrenado nuestro modelo, procedemos a evaluar su desempeño en el conjunto de imágenes de prueba:

In [0]:
prueba_ds = test.map(carga_una_imagen, num_parallel_calls=4)
prueba_ds = prueba_ds.batch(1)

exactitud_prueba = tf.keras.metrics.SparseCategoricalAccuracy()
for i, (x, y) in enumerate(prueba_ds):
  y_pred = paso_prueba(x, modelo)
  exactitud_prueba(y, y_pred)

exactitud_prueba = exactitud_prueba.result().numpy() * 100
print("Exactitud en validación: {:02.2f}%".format(exactitud_prueba))

Finalmente, visualizamos algunos ejemplos de predicciónes hechas por el modelo en el conjunto de prueba.

In [0]:
fig = plt.figure(figsize=(10, 4))
for i, (x, y) in enumerate(prueba_ds, start=1):
    ax = fig.add_subplot(2, 5, i)
    ax.imshow(x[0])
    y_pred = np.argmax(modelo(x))
    ax.imshow(x[0])
    text = 'V:' + str(y.numpy()[0]) + '  P:' + str(y_pred) 
    ax.set_title(text)
    ax.set_xticks([])
    ax.set_yticks([])
    if i == 10:
        break